# NNodely Documentation - Export Network

In [1]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *
from nnodely.relation import NeuObj

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-- nnodely_v1.3.1 --<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [ ]:
x = Input('x')
y = Input('y')
z = Input('z')

def myFun(K1,p1,p2):
    return K1*p1*p2

K_x = Parameter('k_x', dimensions=1, tw=1)
c_v = Constant('c_v', tw=1, values=[[1],[2]])
parfun_x = ParamFun(myFun, parameters_and_constants = [K_x,c_v])

out = Output('out', Fir(parfun_x(x.tw(1))))

result_path = './results'
model = Modely(workspace=result_path)
model.addModel('model', out)
model.neuralizeModel()

## Save network parameters in a pickle file

In [ ]:
model.saveTorchModel(name='model_parameters', model_folder=result_path)

## Load network parameters from a pickle file

In [ ]:
model.loadTorchModel(name='model_parameters', model_folder=result_path)

## Save the model definition in a json file.

In [ ]:
model.saveModel(name='model_definition', model_folder=result_path)

## Load the model definition from a json file

In [ ]:
model.loadModel(name='model_definition', model_folder=result_path)

## Export pytorch model

Exports the neural network model as a stand-alone PyTorch Module class.

In [ ]:
model.exportPythonModel(name='pytorch_model', model_folder=result_path)

## Import pytorch model inside the NNodely framework

In [ ]:
model.importPythonModel(name='pytorch_model', model_folder=result_path)

## Export the network to an ONNX format

In [ ]:
model.exportONNX(name='model_onnx', model_folder=result_path)

You can also specify the input and output order inside the network

In [ ]:
model.exportONNX(name='model_onnx', model_folder=result_path, inputs_order=['x', 'y', 'z'], outputs_order=['out'])

## ONNX inference

You can use a previously exported onnx model to make inference. pass the inputs and the path to the model to run a prediction process.

NOTE: All the states variables that are responsible for the recurrent loop must have an extra dimension (4 dimensions) at the beginning corresponding to the number of prediction samples.

In [ ]:
import numpy as np
data = {'vel':np.random.rand(3,1,1).astype(np.float32), 
        'brk':np.random.rand(5,3,25,1).astype(np.float32), 
        'gear':np.random.rand(5,3,1,1).astype(np.float32), 
        'trq':np.random.rand(5,3,25,1).astype(np.float32), 
        'alt':np.random.rand(5,3,2,21).astype(np.float32)}
inference = Modely().onnxInference(data, result_path)
print(inference)

## Export training and validation report

Generates a PDF report with plots containing the results of the training and validation of the neural network.

In [ ]:
model.exportReport(name='model_report', model_folder=result_path)

## Example - Longitudinal dynamics of a vehicle (Non recurrent version export)

In [ ]:
import os
vehicle = nnodely(visualizer=MPLVisualizer(),seed=2, workspace=os.path.join(os.getcwd(), 'results'))

# Dimensions of the layers
n  = 25
na = 21

#Create neural model inputs
velocity = Input('vel')
brake = Input('brk')
gear = Input('gear')
torque = Input('trq')
altitude = Input('alt',dimensions=na)
acc = Input('acc')

# Create neural network relations
air_drag_force = Linear(b=True)(velocity.last()**2)
breaking_force = -Relu(Fir(parameter_init = init_negexp, parameter_init_params={'size_index':0, 'first_value':0.002, 'lambda':3})(brake.sw(n)))
gravity_force = Linear(W_init=init_constant, W_init_params={'value':0}, dropout=0.1, W='gravity')(altitude.last())
fuzzi_gear = Fuzzify(6, range=[2,7], functions='Rectangular')(gear.last())
local_model = LocalModel(input_function=lambda: Fir(parameter_init = init_negexp, parameter_init_params={'size_index':0, 'first_value':0.002, 'lambda':3}))
engine_force = local_model(torque.sw(n), fuzzi_gear)

# Create neural network output
out = Output('accelleration', air_drag_force+breaking_force+gravity_force+engine_force)

# Add the neural model to the nnodely structure and neuralization of the model
vehicle.addModel('acc',[out])
vehicle.addMinimize('acc_error', acc.last(), out, loss_function='rmse')
vehicle.neuralizeModel(0.05)

## Export the Onnx Model
vehicle.exportONNX(['vel','brk','gear','trq','alt'],['accelleration'])

## Example - Longitudinal dynamics of a vehicle (Recurrent version export)

In [ ]:
vehicle = nnodely(visualizer=MPLVisualizer(),seed=2, workspace=os.path.join(os.getcwd(), 'results'))

# Dimensions of the layers
n  = 25
na = 21

#Create neural model inputs
velocity = State('vel')
brake = Input('brk')
gear = Input('gear')
torque = Input('trq')
altitude = Input('alt',dimensions=na)
acc = Input('acc')

# Create neural network relations
air_drag_force = Linear(b=True)(velocity.last()**2)
breaking_force = -Relu(Fir(W_init = init_negexp, W_init_params={'size_index':0, 'first_value':0.002, 'lambda':3})(brake.sw(n)))
gravity_force = Linear(W_init=init_constant, W_init_params={'value':0}, dropout=0.1, W='gravity')(altitude.last())
fuzzi_gear = Fuzzify(6, range=[2,7], functions='Rectangular')(gear.last())
local_model = LocalModel(input_function=lambda: Fir(W_init = init_negexp, W_init_params={'size_index':0, 'first_value':0.002, 'lambda':3}))
engine_force = local_model(torque.sw(n), fuzzi_gear)

sum_rel = air_drag_force+breaking_force+gravity_force+engine_force

# Closing the loop
sum_rel.closedLoop(velocity)

# Create neural network output
out1 = Output('accelleration', sum_rel)
out2 = Output('velout', altitude.sw(2))

# Add the neural model to the nnodely structure and neuralization of the model
vehicle.addModel('acc',[out1,out2])
vehicle.addMinimize('acc_error', acc.last(), out1, loss_function='rmse')
vehicle.neuralizeModel(0.05)

data = {'vel':np.random.rand(1,1), 'brk':np.random.rand(25,1), 'gear':np.random.rand(1,1), 'trq':np.random.rand(25,1), 'alt':np.random.rand(2,21), 'acc':np.random.rand(1,1)}
inference = vehicle(data)

## Export the Onnx Model
vehicle.exportONNX(['brk','gear','trq','alt','vel'],['accelleration','velout'])

In [ ]:
## Make inference using the onnx model
data = {'vel':np.random.rand(3,1,1).astype(np.float32), 'brk':np.random.rand(5,3,25,1).astype(np.float32), 'gear':np.random.rand(5,3,1,1).astype(np.float32), 'trq':np.random.rand(5,3,25,1).astype(np.float32), 'alt':np.random.rand(5,3,2,21).astype(np.float32)}
print(Modely().onnxInference(data,'results/onnx/net.onnx'))